In [1]:
#!pip install tqdm
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [2]:
#inheriting from modules
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,50)
        self.fc2=nn.Linear(50,num_classes)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x

In [3]:
#setup device
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#define model_parameters
input_size=784
num_classes=10
learning_rate=0.005
batch_size=64
num_epochs=3

In [5]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [20]:
#Load The Data
train_data=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

In [8]:
model=NN(input_size=input_size,num_classes=num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [10]:

for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
        data=data.to(device=device)
        targets=targets.to(device=device)
        
        data=data.reshape(data.shape[0],-1)
        
        scores=model(data)
        loss=criterion(scores,targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:05<00:00, 178.69it/s]


In [17]:
def check_acc(loader,model):
    correct=0
    samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device=device)
            y=y.to(device=device)
            x=x.reshape(x.shape[0],-1)
            
            scores=model(x)
            _,predictions=scores.max(1)
            correct += (predictions == y).sum()
            samples += predictions.size(0)
    model.train()
    return correct/samples

In [18]:
#Training Accuracy
check_acc(train_loader,model)

tensor(0.9768, device='cuda:0')

In [21]:
#Test Accuracy
check_acc(test_loader,model)

tensor(0.9687, device='cuda:0')